In [1]:
#importing the needed libraries
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_validate, KFold
import numpy as np
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, make_scorer, recall_score, f1_score, confusion_matrix, accuracy_score

In [2]:
#reading in the data we will use in the machine learning pipeline, with the top ten features of the method
df = pd.read_excel('5fs_impedance_training_set_10_final.xlsx')
df

,ImP_exp_diff_mean,ImP_exp_dur_sd,ImP_RRV_RMSSD,ImP_insp_flow_sd,ImP_ie_ratio_sd,ImP_duty_cycle_mean,ImP_insp_amp_sd,ImP_RSP_Symmetry_PeakTrough,ImP_exp_amp_sd,ImP_exp_dur_mean,Task_Label,Participant,Classification
0,0.118857,0.171499,382.434308,0.015663,0.048638,0.489158,0.050985,0.517348,0.046774,1.694000,12.0a,10785,0
1,0.225143,0.173505,390.361005,0.020811,0.069124,0.485452,0.030734,0.528718,0.031616,1.748000,12.0b,10785,0
2,0.236000,0.222699,434.669530,0.016278,0.050025,0.476758,0.031615,0.520548,0.026435,2.272000,12.0c,10785,0
3,0.250667,0.190379,403.425334,0.015714,0.071244,0.483507,0.029114,0.530733,0.025312,1.931429,12.0d,10785,0
4,0.140800,0.174310,483.103302,0.006162,0.124387,0.481444,0.043006,0.527149,0.028574,2.132000,12.0e,10785,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2569,0.240000,0.274684,318.223192,0.003850,0.265283,0.489451,0.008020,0.524840,0.014866,1.499600,85,98586,0
2570,0.708000,0.764264,1236.868088,0.072599,0.375168,0.427189,0.121972,0.511842,0.102499,2.301714,87,98586,1
2571,0.119556,0.122849,159.285907,0.004550,0.082769,0.499069,0.006324,0.543171,0.012527,1.433600,89,98586,0
2572,1.567500,0.980966,2899.130116,0.021556,0.546540,0.482030,0.047870,0.472444,0.051081,1.872000,91,98586,1


In [3]:
# extracting the features from the DataFrame
features = df.drop(columns=['Classification', 'Participant', 'Task_Label'])

# extracting the Classification (0/1) and participants from the dataframe
target = df['Classification'].values
participants = df['Participant'].values
task = df['Task_Label'].values

# extracting the feature names
feature_names = features.columns.tolist()

# converting the features, target, participants to numpy arrays
features_array = features.values
target_array = target

# creating the dictionary with the needed format to enter into the nested cross-validation
data_dict = {
    'data': features_array,
    'target': target_array,
    'feature_names': feature_names,
    'participants': participants,
    'task': task
}

data_dict

{'data': array([[1.18857100e-01, 1.71499300e-01, 3.82434308e+02, ...,
         5.17347900e-01, 4.67743000e-02, 1.69400000e+00],
        [2.25142900e-01, 1.73505000e-01, 3.90361005e+02, ...,
         5.28718100e-01, 3.16159000e-02, 1.74800000e+00],
        [2.36000000e-01, 2.22698600e-01, 4.34669530e+02, ...,
         5.20547800e-01, 2.64352000e-02, 2.27200000e+00],
        ...,
        [1.19555600e-01, 1.22848900e-01, 1.59285907e+02, ...,
         5.43171200e-01, 1.25266000e-02, 1.43360000e+00],
        [1.56750000e+00, 9.80965500e-01, 2.89913012e+03, ...,
         4.72444400e-01, 5.10806000e-02, 1.87200000e+00],
        [2.16500000e-01, 1.61567600e-01, 3.19002015e+02, ...,
         5.20748900e-01, 1.69911000e-02, 1.65511110e+00]]),
 'target': array([0, 0, 0, ..., 0, 1, 0]),
 'feature_names': ['ImP_exp_diff_mean',
  'ImP_exp_dur_sd',
  'ImP_RRV_RMSSD',
  'ImP_insp_flow_sd',
  'ImP_ie_ratio_sd',
  'ImP_duty_cycle_mean',
  'ImP_insp_amp_sd',
  'ImP_RSP_Symmetry_PeakTrough',
  'ImP_exp_am

In [4]:
#running the nested cross-validation for the Impedance (with VU-AMS 5fs) method using Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score

# random seed for reproducibility
random_seed = 1
np.random.seed(random_seed)

X = data_dict['data']
Y = data_dict['target']
groups = data_dict['participants']

# setting up the parameter grid: the combinations of these will be tried out for ever outer loop's inner loop
param_grid = {
    'penalty' : ['l1', 'l2'],
    'C' : (1, 4, 20),
    'solver' : ['liblinear','lbfgs']
}


lrc = LogisticRegression(random_state=random_seed)

logo = LeaveOneGroupOut()


# defining the techniques to use for the inner and outer loops
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1)
outer_cv = logo.split(X, Y, groups)

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1_score': make_scorer(f1_score, average='macro'),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True, average='weighted')
}

# performing nested cross-validation
clf = GridSearchCV(estimator=lrc, param_grid=param_grid, cv=inner_cv)
cv_dic = cross_validate(clf, X, Y, cv=outer_cv, scoring=scoring, return_estimator=True, return_train_score=False)
mean_acc_score = cv_dic['test_accuracy'].mean()
mean_prec_score = cv_dic['test_precision'].mean()
mean_rec_score = cv_dic['test_recall'].mean()
mean_f1_score = cv_dic['test_f1_score'].mean()
mean_roc_auc = cv_dic['test_roc_auc'].mean()  # Mean ROC AUC score


print('nested_accuracy_scores:   ', cv_dic['test_accuracy'])
print('mean score:            {0:.4f}'.format(mean_acc_score))

print('nested_precision_scores:   ', cv_dic['test_precision'])
print('mean score:            {0:.4f}'.format(mean_prec_score))

print('nested_recall_scores:   ', cv_dic['test_recall'])
print('mean score:            {0:.4f}'.format(mean_rec_score))

print('nested_f1_scores:   ', cv_dic['test_f1_score'])
print('mean score:            {0:.4f}'.format(mean_f1_score))

print('nested_roc_auc_scores:   ', cv_dic['test_roc_auc'])
print('mean score:            {0:.4f}'.format(mean_roc_auc))

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

...

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

nested_accuracy_scores:    [0.92424242 0.89393939 0.83333333 0.84848485 0.87878788 0.93939394
 0.95454545 0.93939394 0.95454545 0.74242424 0.90909091 0.96969697
 0.93939394 0.87878788 0.98484848 0.87878788 0.89393939 0.89393939
 0.90909091 0.71212121 0.89393939 0.90909091 0.90909091 0.92424242
 0.92424242 0.77272727 0.92424242 0.90909091 0.83333333 0.81818182
 0.95454545 0.87878788 0.92424242 1.         0.93939394 0.93939394
 0.93939394 0.86363636 0.92424242]
mean score:            0.8990
nested_precision_scores:    [0.93230174 0.90909091 0.84033613 0.85353535 0.89818182 0.94008264
 0.95959596 0.94008264 0.95959596 0.75110075 0.92727273 0.97107438
 0.93939394 0.87878788 0.98520085 0.89818182 0.9000259  0.9000259
 0.92045455 0.72131572 0.9000259  0.92045455 0.91223776 0.92981093
 0.92398198 0.77754011 0.92981093 0.90909091 0.83512195 0.83787879
 0.95959596 0.88699495 0.92611833 1.         0.93939394 0.93939394
 0.94213287 0.87782767 0.93230174]
mean score:            0.9053
nested_recal

/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/melisa/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

In [5]:
sensitivity_scores = []
specificity_scores = []

# iteration over the outer folds to calculate average sensitivity and specificity
for i, (train_index, test_index) in enumerate(logo.split(X, Y, groups)):
    print(f"Fold {i + 1}:")

    # getting the trained estimator for the present fold
    estimator = cv_dic['estimator'][i]
    
    # predicting on the test set
    y_pred = estimator.predict(X[test_index])
    
    # creating confusion matrix
    tn, fp, fn, tp = confusion_matrix(Y[test_index], y_pred).ravel()
    
    # calculating the sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    print(f"Sensitivity: {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print("=" * 30)
    
    # store the sensitivity and specificity scores
    sensitivity_scores.append(sensitivity)
    specificity_scores.append(specificity)

    
    # calculate and print the averages of the sensitivity and specificity scores previously calculated
average_sensitivity = np.mean(sensitivity_scores)
average_specificity = np.mean(specificity_scores)

print("Average Sensitivity:", average_sensitivity)
print("Average Specificity:", average_specificity)

Fold 1:
Sensitivity: 0.7917
Specificity: 1.0000
Fold 2:
Sensitivity: 0.7083
Specificity: 1.0000
Fold 3:
Sensitivity: 0.6250
Specificity: 0.9524
Fold 4:
Sensitivity: 0.6667
Specificity: 0.9524
Fold 5:
Sensitivity: 0.6667
Specificity: 1.0000
Fold 6:
Sensitivity: 0.8750
Specificity: 0.9762
Fold 7:
Sensitivity: 1.0000
Specificity: 0.9286
Fold 8:
Sensitivity: 0.8750
Specificity: 0.9762
Fold 9:
Sensitivity: 1.0000
Specificity: 0.9286
Fold 10:
Sensitivity: 0.7083
Specificity: 0.7619
Fold 11:
Sensitivity: 1.0000
Specificity: 0.8571
Fold 12:
Sensitivity: 0.9167
Specificity: 1.0000
Fold 13:
Sensitivity: 0.9167
Specificity: 0.9524
Fold 14:
Sensitivity: 0.8333
Specificity: 0.9048
Fold 15:
Sensitivity: 0.9583
Specificity: 1.0000
Fold 16:
Sensitivity: 0.6667
Specificity: 1.0000
Fold 17:
Sensitivity: 0.9167
Specificity: 0.8810
Fold 18:
Sensitivity: 0.9167
Specificity: 0.8810
Fold 19:
Sensitivity: 0.7500
Specificity: 1.0000
Fold 20:
Sensitivity: 0.6667
Specificity: 0.7381
Fold 21:
Sensitivity: 0.9167


In [6]:
#print all performance scores that might be of interest
import scipy.stats

def calculate_stats(data):
    mean = np.mean(data)
    sd = np.std(data, ddof=1)  # using sample standard deviation (N-1)
    ci = scipy.stats.norm.interval(0.95, loc=mean, scale=sd/np.sqrt(len(data)))
    min_val = np.min(data)
    max_val = np.max(data)
    return mean, sd, ci, min_val, max_val

keys = ['test_accuracy', 'test_precision', 'test_recall', 'test_f1_score', 'test_roc_auc', 'sensitivity_scores', 'specificity_scores']

for key in keys:
    if key in cv_dic:
        data = cv_dic[key]
    elif key == 'sensitivity_scores':
        data = sensitivity_scores
    elif key == 'specificity_scores':
        data = specificity_scores

    stats = calculate_stats(data)
    print(f"{key}: Mean={stats[0]}, SD={stats[1]}, 95% CI={stats[2]}, Min={stats[3]}, Max={stats[4]}")


test_accuracy: Mean=0.898989898989899, SD=0.0611353812289848, 95% CI=(0.87980283993549, 0.918176958044308), Min=0.7121212121212122, Max=1.0
test_precision: Mean=0.9052654043821954, SD=0.05905961674045037, 95% CI=(0.8867298144938226, 0.9238009942705682), Min=0.7213157213157212, Max=1.0
test_recall: Mean=0.898989898989899, SD=0.0611353812289848, 95% CI=(0.87980283993549, 0.918176958044308), Min=0.7121212121212122, Max=1.0
test_f1_score: Mean=0.888975339489796, SD=0.06727155543498024, 95% CI=(0.8678624702990668, 0.9100882086805252), Min=0.6964415395787946, Max=1.0
test_roc_auc: Mean=0.9577991452991454, SD=0.052115196997152655, 95% CI=(0.9414430297026593, 0.9741552608956316), Min=0.742063492063492, Max=1.0
sensitivity_scores: Mean=0.8429487179487177, SD=0.12553305145875726, 95% CI=(0.8035507453966565, 0.882346690500779), Min=0.5, Max=1.0
specificity_scores: Mean=0.9310134310134308, SD=0.06817897820755091, 95% CI=(0.9096157713466364, 0.9524110906802252), Min=0.7380952380952381, Max=1.0


In [7]:
# IF this is the best-performing nested CV for the method: print best hyperparameters for each outer loop
for fold_idx, estimator in enumerate(cv_dic['estimator']):
    print(f'Fold {fold_idx + 1} - Best hyperparameters: {estimator.best_params_}')


Fold 1 - Best hyperparameters: {'C': 4, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 2 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 3 - Best hyperparameters: {'C': 4, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 4 - Best hyperparameters: {'C': 4, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 5 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 6 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 7 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 8 - Best hyperparameters: {'C': 4, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 9 - Best hyperparameters: {'C': 4, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 10 - Best hyperparameters: {'C': 20, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 11 - Best hyperparameters: {'C': 4, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 12 - Best hyperparameters: {'C': 4, 'penalty': 'l1', 'solver': 'liblinear'}
Fold 13 - Best hyperparameters: 

In [8]:
# IF this is the best-performing nested CV for the method: print mode of hyperparameters

from collections import defaultdict, Counter

# initialize a dictionary to contain all hyperparameters and their values across loops
hyperparameters = defaultdict(list)

# extracting hyperparameters of each outer loop
for estimator in cv_dic['estimator']:
    for param, value in estimator.best_params_.items():
        hyperparameters[param].append(value)

# count the most frequent value for each hyperparameter
for param, values in hyperparameters.items():
    most_common_value, count = Counter(values).most_common(1)[0]  # Get the most frequent value
    print(f'Hyperparameter: {param} - Most frequent value: {most_common_value} (Count: {count})')


Hyperparameter: C - Most frequent value: 4 (Count: 23)
Hyperparameter: penalty - Most frequent value: l1 (Count: 36)
Hyperparameter: solver - Most frequent value: liblinear (Count: 39)
